# マルチインスタンス学習（クラス9が正）のノートブック

In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
# ハイパーパラメータ
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2
BAG_SIZE = 4

In [3]:
# データセットの読み込み
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
# 9を正例とみなす
y_train = (y_train == 9).astype(int)
y_val = (y_val == 9).astype(int)
y_test = (y_test == 9).astype(int)

In [5]:
# マルチインスタンスの形式に変換（バッグ内のインスタンス数はBAG_SIZE）
X_train, y_train = rewrite_label_with_mil_setting(X_train, y_train, BAG_SIZE)

In [6]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):
    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train.reshape(-1, N_COMPONENTS),
                             label=np.zeros(len(X_train.reshape(-1, N_COMPONENTS))), free_raw_data=False)
    train_data._mil_labels = np.repeat(y_train, BAG_SIZE)
    # 適当な初期化。本当はラベルの割合などを用いて正規化して推定するべき
    train_data._weight_s0 = np.stack([np.repeat(np.ones_like(y_train) * (y_train == 0), BAG_SIZE),
                                      np.repeat(np.zeros_like(y_train) * (y_train == 0), BAG_SIZE)], axis=-1)
    train_data._weight_s1 = np.stack([np.repeat(np.ones_like(y_train) * (y_train == 1), BAG_SIZE),
                                      np.repeat(np.ones_like(y_train) * (y_train == 1), BAG_SIZE)], axis=-1)
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "custom",
        "metric": "custom",
        "verbose": -1,
        "learning_rate": 0.1,
        "num_boost_round": 200,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        fobj=lambda *x: mil_loss_objective(*x, bag_size=BAG_SIZE),
        feval=binary_metric,
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [7]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.90425
0.90425
0.90425
0.9236666666666666
0.9236666666666666
0.925
0.9238333333333333
0.928
0.92925
0.9291666666666667
0.93
0.93175
0.9326666666666666
0.9339166666666666
0.9363333333333334
0.9375833333333333
0.9393333333333334
0.9399166666666666
0.9403333333333334
0.9415833333333333
0.94275
0.94325
0.94475
0.9458333333333333
0.9475
0.9485
0.9491666666666667
0.94975
0.95075
0.9515
0.9525
0.9536666666666667
0.9541666666666667
0.955
0.9553333333333334
0.95625
0.9565
0.9570833333333333
0.9580833333333333
0.9586666666666667
0.9589166666666666
0.95925
0.9599166666666666
0.9603333333333334
0.9608333333333333
0.9614166666666667
0.9619166666666666
0.9625
0.9630833333333333
0.9638333333333333
0.9641666666666666
0.9646666666666667
0.965
0.9653333333333334
0.9659166666666666
0.9661666666666666
0.9663333333333334
0.9670833333333333
0.9673333333333334
0.9675
0.9678333333333333
0.9680833333333333
0.9685
0.9688333333333333
0.9693333333333334
0.9696666666666667
0.9701666666666666
0.9705
0.970833333333

In [11]:
np.save("mil_valid_accuracies.npy", np.array(valid_accuracies))